<a href="https://colab.research.google.com/github/afsanasinthi/cse713/blob/main/desenet_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.resnet152V2 import ResNet152V2
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
batch_size = 32
epochs = 50
train_path = '/content/drive/MyDrive/SunflowerDiseaseClassification/new_dataset_v1/train'
valid_path = '/content/drive/MyDrive/SunflowerDiseaseClassification/new_dataset_v1/val'

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
# import tensorflow
# InceptionV3 =tensorflow.keras.applications.inception_v3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
densenet121=DenseNet121(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights
for layer in densenet121.layers:
    layer.trainable = False

In [ ]:
  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/SunflowerDiseaseClassification/new_dataset_v1/train/*')

In [ ]:
# our layers - you can add more if you want
x = Flatten()(densenet121.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/SunflowerDiseaseClassification/new_dataset_v1/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/SunflowerDiseaseClassification/new_dataset_v1/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
train_datagen = ImageDataGenerator()
training_set = train_datagen.flow_from_directory(directory='/content/drive/MyDrive/SunflowerDiseaseClassification/new_dataset_v1/train',target_size=(224,224), shuffle=False, class_mode='categorical')
test_datagen = ImageDataGenerator()
test_set = test_datagen.flow_from_directory(directory='/content/drive/MyDrive/SunflowerDiseaseClassification/new_dataset_v1/val', target_size=(224,224),shuffle=False, class_mode='categorical')

In [ ]:
training_steps_per_epoch = np.ceil(training_set.samples / batch_size)
validation_steps_per_epoch = np.ceil(test_set.samples / batch_size)
r=model.fit_generator(training_set, steps_per_epoch = training_steps_per_epoch, validation_data=test_set, validation_steps=validation_steps_per_epoch,epochs=epochs, verbose=1)
print('Training Completed!')

In [ ]:
Y_pred = model.predict(test_set, test_set.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
import sklearn.metrics as metrics
val_trues =test_set.classes
from sklearn.metrics import classification_report
print(classification_report(val_trues, val_preds))

In [ ]:
# fit the model
# Run the cell. It will take some time to execute


training_steps_per_epoch = np.ceil(training_set.samples / batch_size)
validation_steps_per_epoch = np.ceil(test_set.samples / batch_size)
# model.fit_generator(training_set, steps_per_epoch = training_steps_per_epoch, validation_data=test_set, validation_steps=validation_steps_per_epoch,epochs=epochs, verbose=1)
# print('Training Completed!')


r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=epochs,
  steps_per_epoch=training_steps_per_epoch,
  validation_steps=validation_steps_per_epoch
)

In [ ]:
Y_pred = model.predict(test_set, test_set.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
import sklearn.metrics as metrics
val_trues =test_set.classes
from sklearn.metrics import classification_report
print(classification_report(val_trues, val_preds))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# import all the metrics we'll use later on
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)


In [ ]:
prediction_RF = model.predict(test_set)


In [ ]:
# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_curve, auc, roc_curve
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,precision_score,recall_score
# from sklearn.metrics.classification import accuracy_score, log_loss
# print('Shape of x_train: ', training_set.shape)
print('Shape of y_train: ', prediction_RF.size)
# print('Shape of x_test : ', xtest.shape)
# print('shape of y_test : ', ytest.shape)
print('Precision score  : ', precision_score(test_set, prediction_RF, average='micro'))
print('Recall score  : ', recall_score(test_set, prediction_RF, average='micro'))
print('f1 score : ', f1_score(test_set, prediction_RF, average='micro'))

In [ ]:
# Generate confusion matrix for the predictions
conf_matrix = confusion_matrix(test_set, prediction_RF)
conf_matrix